In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/uhack-sentiments-20-decode-code-words/Dataset Description.xlsx
/kaggle/input/uhack-sentiments-20-decode-code-words/Participants_Data_DCW/submission.csv
/kaggle/input/uhack-sentiments-20-decode-code-words/Participants_Data_DCW/train.csv
/kaggle/input/uhack-sentiments-20-decode-code-words/Participants_Data_DCW/test.csv


In [2]:
import os
import gc
import re
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
def random_seed(seed_value):
    import random 
    random.seed(seed_value)  
    import numpy as np
    np.random.seed(seed_value) 
    import torch
    torch.manual_seed(seed_value) 
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True   
        torch.backends.cudnn.benchmark = False
        
random_seed(42)

In [4]:
train = pd.read_csv('/kaggle/input/uhack-sentiments-20-decode-code-words/Participants_Data_DCW/train.csv')
test = pd.read_csv('/kaggle/input/uhack-sentiments-20-decode-code-words/Participants_Data_DCW/test.csv')
sub = pd.read_csv('/kaggle/input/uhack-sentiments-20-decode-code-words/Participants_Data_DCW/submission.csv')
train

,Id,Review,Components,Delivery and Customer Support,Design and Aesthetics,Dimensions,Features,Functionality,Installation,Material,Price,Quality,Usability,Polarity
0,0,For some reason everybody complains and I'm co...,0,0,0,0,0,0,0,0,0,0,1,0
1,1,"I like everything about it, great choice of sp...",0,0,0,0,1,1,0,0,0,0,0,1
2,2,Excellent ceiling fan brace. Easy to install a...,0,0,0,0,0,0,1,0,0,1,0,1
3,3,Work great easy to use . No issues at all with...,0,0,0,0,0,1,0,0,0,0,1,1
4,4,I would recommend this product because it is p...,0,0,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6131,6131,Easy to install. Just be careful with the razo...,0,0,0,0,0,0,1,0,0,0,0,1
6132,6132,Got a big pup and she had to learn about scree...,0,0,0,0,0,0,0,0,0,1,1,1
6133,6133,We installed our Nest cam system a little over...,0,0,0,0,0,0,0,0,0,1,1,1
6134,6134,this is a below average camera . The only thin...,0,0,0,0,0,0,0,0,1,0,0,0


In [5]:
target = ['Components', 'Delivery and Customer Support',
       'Design and Aesthetics', 'Dimensions', 'Features', 'Functionality',
       'Installation', 'Material', 'Price', 'Quality', 'Usability',
       'Polarity']

In [6]:


# FREQWORDS = set([w for (w, wc) in cnt.most_common(50)])
# def remove_freqwords(text):
#     """custom function to remove the frequent words"""
#     return " ".join([word for word in str(text).split() if word not in FREQWORDS])

# df["Review"] = df["Review"].apply(lambda text: remove_freqwords(text))
# df.head()




# n_rare_words = 100
# RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])


In [7]:
for t in target:
    print(train[t].value_counts()/train.shape[0])

0    0.955346
1    0.044654
Name: Components, dtype: float64
0    0.970828
1    0.029172
Name: Delivery and Customer Support, dtype: float64
0    0.894068
1    0.105932
Name: Design and Aesthetics, dtype: float64
0    0.886734
1    0.113266
Name: Dimensions, dtype: float64
0    0.94899
1    0.05101
Name: Features, dtype: float64
0    0.586375
1    0.413625
Name: Functionality, dtype: float64
0    0.875815
1    0.124185
Name: Installation, dtype: float64
0    0.974739
1    0.025261
Name: Material, dtype: float64
0    0.860821
1    0.139179
Name: Price, dtype: float64
0    0.645209
1    0.354791
Name: Quality, dtype: float64
0    0.799707
1    0.200293
Name: Usability, dtype: float64
1    0.777379
0    0.222621
Name: Polarity, dtype: float64


In [8]:
# def get_df(df,target):
# train = train[train[target].duplicated()==False].copy()

In [9]:
# train[train['Usability']==1][target].value_counts()/


In [10]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# tarMap={'Depression':0,'Alcohol':1,'Suicide':2,'Drugs':3}
# train.label=train.label.map(tarMap)
# train

In [12]:
# df=train.append(test,ignore_index=True)
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
punctuation=string.punctuation
from bs4 import BeautifulSoup
# !pip install emoji
import emoji
import string
punctuation=string.punctuation

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
from wordcloud import WordCloud, STOPWORDS
stopwords1 = set(STOPWORDS)
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.utils import shuffle



def url_to_words(raw_text):
    raw_text=str(raw_text).strip()
    soup = BeautifulSoup(raw_text, 'html.parser')
    raw_text = soup.text
    raw_text = emoji.demojize(raw_text)
    no_coms=re.sub(r'\.com','',raw_text)
    no_urls=re.sub('https?://www','',no_coms)
    no_urls1=re.sub('https?://','',no_urls)
    try:
        no_encoding=no_urls1.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        no_encoding = no_urls1
    letters_only = re.sub("[^a-zA-Z0-9]", " ",no_urls1) 
    words = letters_only.split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w.lower() for w in words if w not in stops] 
    return( " ".join( meaningful_words ))


# train['Review'] = train['Review'].apply(url_to_words)
# test['Review'] = test['Review'].apply(url_to_words)

In [14]:
train.sample()

,Id,Review,Components,Delivery and Customer Support,Design and Aesthetics,Dimensions,Features,Functionality,Installation,Material,Price,Quality,Usability,Polarity
4290,4290,"metal is best, plastic deteriorates. worth the...",0,0,0,0,0,0,0,1,1,0,0,1


In [15]:
# df.columns



In [16]:
train['Components'].value_counts()

0    5862
1     274
Name: Components, dtype: int64

In [17]:
df=train.append(test,ignore_index=True)
df

,Id,Review,Components,Delivery and Customer Support,Design and Aesthetics,Dimensions,Features,Functionality,Installation,Material,Price,Quality,Usability,Polarity
0,0,For some reason everybody complains and I'm co...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,"I like everything about it, great choice of sp...",0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2,Excellent ceiling fan brace. Easy to install a...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,3,Work great easy to use . No issues at all with...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
4,4,I would recommend this product because it is p...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8762,2626,Very strong piece of hardware. Easy to adjust ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8763,2627,Great supot. Square Wich is better to line up...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8764,2628,"No jams, no problems. Good quality nail!",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8765,2629,"Chair cushion is and very firm, however nice r...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df["Review_len"] = df['Review'].apply(len)

# extract features from text
df['word_count'] = df['Review'].apply(lambda x: len(x.split()))
df['word_density'] = df['Review_len'] / (df['word_count']+1)
df['punctuation_count'] = df['Review'].apply(lambda x: len("".join(_ for _ in x if _ in punctuation))) 
df['title_word_count'] = df['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
df['upper_case_word_count'] = df['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))
df['stopword_count'] = df['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.lower() in stopwords1]))

from textblob import TextBlob

pos_dic = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def pos_check(x, flag):
    cnt = 0
    try:
        wiki = TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_dic[flag]:
                cnt += 1
    except:
        pass
    return cnt

df['noun_count'] = df['Review'].apply(lambda x: pos_check(x, 'noun'))
df['verb_count'] = df['Review'].apply(lambda x: pos_check(x, 'verb'))
df['adj_count'] = df['Review'].apply(lambda x: pos_check(x, 'adj'))
df['adv_count'] = df['Review'].apply(lambda x: pos_check(x, 'adv'))
df['pron_count'] = df['Review'].apply(lambda x: pos_check(x, 'pron'))

In [19]:
df.head()

,Id,Review,Components,Delivery and Customer Support,Design and Aesthetics,Dimensions,Features,Functionality,Installation,Material,...,word_density,punctuation_count,title_word_count,upper_case_word_count,stopword_count,noun_count,verb_count,adj_count,adv_count,pron_count
0,0,For some reason everybody complains and I'm co...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.735849,4,4,2,28,13,10,2,7,8
1,1,"I like everything about it, great choice of sp...",0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,4.750000,3,1,1,11,7,2,2,0,4
2,2,Excellent ceiling fan brace. Easy to install a...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,5.272727,1,2,0,2,4,2,1,1,0
3,3,Work great easy to use . No issues at all with...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,4.142857,1,2,0,5,3,2,2,0,0
4,4,I would recommend this product because it is p...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.921053,2,1,1,22,5,10,3,0,4


In [20]:
!pip install vaderSentiment


from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
#     print(score)
    return score['neg'],score['pos'],score['neu'],score['compound']
    
df['sentiment'] = df['Review'].apply(sentiment_analyzer_scores)


df['sent_neg'] = df['sentiment'].apply(lambda x: x[0])
df['sent_pos'] = df['sentiment'].apply(lambda x: x[1])
df['sent_neu'] = df['sentiment'].apply(lambda x: x[2])
df['sent_comp'] = df['sentiment'].apply(lambda x: x[3])
df.drop(['sentiment'],axis=1,inplace=True)

     |████████████████████████████████| 125 kB 399 kB/s            


In [21]:
df['Review'] = df['Review'].apply(url_to_words)

In [22]:
from collections import Counter
cnt = Counter()
for text in df["Review"].values:
    for word in text.split():
        cnt[word] += 1
        
print(cnt.most_common(50))


# FREQWORDS = set([w for (w, wc) in cnt.most_common(50)])
# def remove_freqwords(text):
#     """custom function to remove the frequent words"""
#     return " ".join([word for word in str(text).split() if word not in FREQWORDS])

# df["Review"] = df["Review"].apply(lambda text: remove_freqwords(text))
# df.head()




n_rare_words = 100
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])

RAREWORDS
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

df["Review"] = df["Review"].apply(lambda text: remove_rarewords(text))
df.head()

[('i', 7458), ('great', 2390), ('easy', 2077), ('the', 1910), ('use', 1499), ('it', 1477), ('well', 1450), ('good', 1406), ('one', 1258), ('works', 1133), ('this', 1090), ('quality', 965), ('would', 958), ('water', 946), ('like', 909), ('product', 897), ('work', 858), ('install', 810), ('price', 766), ('worked', 761), ('screen', 719), ('used', 656), ('get', 604), ('toilet', 602), ('they', 601), ('made', 581), ('love', 567), ('bought', 558), ('time', 556), ('sprinkler', 552), ('very', 549), ('box', 529), ('tool', 516), ('recommend', 513), ('nails', 504), ('nice', 501), ('camera', 500), ('perfect', 488), ('nozzle', 487), ('cut', 486), ('2', 475), ('clamps', 471), ('these', 446), ('really', 439), ('see', 438), ('much', 428), ('buy', 417), ('also', 417), ('little', 416), ('better', 413)]


,Id,Review,Components,Delivery and Customer Support,Design and Aesthetics,Dimensions,Features,Functionality,Installation,Material,...,stopword_count,noun_count,verb_count,adj_count,adv_count,pron_count,sent_neg,sent_pos,sent_neu,sent_comp
0,0,for reason everybody complains i complaining t...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,28,13,10,2,7,8,0.129,0.046,0.825,-0.5849
1,1,i like everything great choice spray patterns ...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,11,7,2,2,0,4,0.000,0.239,0.761,0.7650
2,2,excellent ceiling fan brace easy install well ...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2,4,2,1,1,0,0.000,0.647,0.353,0.8750
3,3,work great easy use no issues hanging fan,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,5,3,2,2,0,0,0.107,0.454,0.439,0.7964
4,4,i would recommend product perfect watering han...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,22,5,10,3,0,4,0.000,0.150,0.850,0.7351


In [23]:

from collections import Counter
target = ['Components', 'Delivery and Customer Support',
       'Design and Aesthetics', 'Dimensions', 'Features', 'Functionality',
       'Installation', 'Material', 'Price', 'Quality', 'Usability',
       'Polarity']
def getfeats(df,tr):
    text_cliqs=[]
    cnt = Counter()
    for text in df[df[tr]==1]['Review']:
        for word in text.split():
            cnt[word] += 1
    for k in cnt.most_common()[:-300-1:-1]:
        if len(k[0])>5:
            text_cliqs.append(k[0])
    return text_cliqs
def get_targ_ft(val,lst):
    c=0
    for v in val.split():
        if v in lst:
            c+=1
    return c
    
    
for k in target:
    
    ls = getfeats(df,k)
    df[k+'_isin'] = df['Review'].apply(get_targ_ft,lst=ls)


In [24]:
# cnt.most_common()

In [25]:
# text_cliqs=[]
# for k in cnt.most_common()[:-200-1:-1]:
#     if len(k[0])>5:
#         text_cliqs.append(k[0])

In [26]:
# text_cliqs
df.columns

Index(['Id', 'Review', 'Components', 'Delivery and Customer Support',
       'Design and Aesthetics', 'Dimensions', 'Features', 'Functionality',
       'Installation', 'Material', 'Price', 'Quality', 'Usability', 'Polarity',
       'Review_len', 'word_count', 'word_density', 'punctuation_count',
       'title_word_count', 'upper_case_word_count', 'stopword_count',
       'noun_count', 'verb_count', 'adj_count', 'adv_count', 'pron_count',
       'sent_neg', 'sent_pos', 'sent_neu', 'sent_comp', 'Components_isin',
       'Delivery and Customer Support_isin', 'Design and Aesthetics_isin',
       'Dimensions_isin', 'Features_isin', 'Functionality_isin',
       'Installation_isin', 'Material_isin', 'Price_isin', 'Quality_isin',
       'Usability_isin', 'Polarity_isin'],
      dtype='object')

In [27]:
cols = ['sent_neu','sent_comp','sent_neg','sent_pos','Review_len', 'word_count', 'word_density', 'punctuation_count',
       'title_word_count', 'upper_case_word_count', 'stopword_count',
        'noun_count', 'verb_count',
       'adj_count', 'adv_count', 'pron_count'
       ,'Components_isin',
       'Delivery and Customer Support_isin', 'Design and Aesthetics_isin',
       'Dimensions_isin', 'Features_isin', 'Functionality_isin',
       'Installation_isin', 'Material_isin', 'Price_isin', 'Quality_isin',
       'Usability_isin', 'Polarity_isin']

df[cols].describe()

,sent_neu,sent_comp,sent_neg,sent_pos,Review_len,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,...,Design and Aesthetics_isin,Dimensions_isin,Features_isin,Functionality_isin,Installation_isin,Material_isin,Price_isin,Quality_isin,Usability_isin,Polarity_isin
count,8767.000000,8767.000000,8767.000000,8767.000000,8767.000000,8767.000000,8767.000000,8767.000000,8767.000000,8767.000000,...,8767.000000,8767.000000,8767.000000,8767.000000,8767.000000,8767.000000,8767.000000,8767.000000,8767.000000,8767.000000
mean,0.681035,0.481357,0.047763,0.271200,161.534048,30.121250,5.107262,4.232120,3.119197,0.981522,...,0.165051,0.170412,0.185696,0.057602,0.201779,0.507357,0.162313,0.066956,0.115547,0.038782
std,0.176145,0.460338,0.081852,0.193946,180.664774,33.983311,0.581419,4.872088,3.049827,2.786186,...,0.552117,0.533971,0.603973,0.323241,0.600028,1.000643,0.546078,0.314980,0.436728,0.258290
min,0.000000,-0.966300,0.000000,0.000000,16.000000,2.000000,3.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.577000,0.318200,0.000000,0.122000,55.000000,10.000000,4.760000,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.714000,0.636900,0.000000,0.236000,101.000000,19.000000,5.096774,3.000000,2.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.816000,0.816100,0.074000,0.392500,200.000000,38.000000,5.428571,5.000000,4.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,0.996700,0.756000,1.000000,3386.000000,649.000000,9.545455,65.000000,48.000000,138.000000,...,14.000000,8.000000,21.000000,8.000000,17.000000,16.000000,12.000000,5.000000,9.000000,7.000000


In [28]:
testdf = df[df['Components'].isnull()==True]
traindf = df[df['Components'].isnull()==False]


In [29]:
# train = train[~train['Review'].duplicated()].reset_index(drop=True)


# traindf['Review'] = traindf['Review'].apply(url_to_words)
# testdf['Review'] = testdf['Review'].apply(url_to_words)



from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.tokenize import TreebankWordTokenizer
# norm = 'l1', lowercase=True, smooth_idf=False, sublinear_tf=False, ngram_range=(1,4), tokenizer=TreebankWordTokenizer().tokenize
v_1 = TfidfVectorizer(max_features=2500,ngram_range=(1,3),stop_words="english", analyzer='word',norm = 'l1', lowercase=False,
                      smooth_idf=False, sublinear_tf=False, tokenizer=TreebankWordTokenizer().tokenize)
typ_tr =v_1.fit_transform(traindf['Review'])
typ_ts =v_1.transform(testdf['Review'])


# v_1c = TfidfVectorizer(ngram_range=(2,5),stop_words="english", analyzer='char',lowercase=False,
#                       smooth_idf=False, sublinear_tf=False)
# typ_trc =v_1c.fit_transform(train['Review'])
# typ_tsc =v_1c.transform(test['Review'])

In [30]:
from scipy.sparse import csr_matrix
from scipy import sparse
final_features = sparse.hstack((traindf[cols],typ_tr)).tocsr()
final_featurest = sparse.hstack((testdf[cols],typ_ts )).tocsr()

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss,mean_squared_log_error

X = final_features
y = train[target]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1994)
test_X = final_featurest

In [32]:
X_train,X_val

(<4908x2528 sparse matrix of type '<class 'numpy.float64'>'
 	with 138687 stored elements in Compressed Sparse Row format>,
 <1228x2528 sparse matrix of type '<class 'numpy.float64'>'
 	with 34165 stored elements in Compressed Sparse Row format>)

In [33]:
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import make_scorer
from scipy import sparse
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,IsolationForest
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier,OneVsOneClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [34]:
# m=SVC(probability=True,C=1, kernel='sigmoid', degree=1, gamma='scale', coef0=1, 
#       shrinking=True,  tol=0.0001, cache_size=500, class_weight='balanced', 
#       verbose=False, max_iter=-1, decision_function_shape='ovr', random_state=1994)



# m=CatBoostClassifier(n_estimators=5000,random_state=1994,
#                      eval_metric='MultiLogloss',learning_rate=0.1,random_strength=200,
#                   bagging_temperature=0.6,
#                   l2_leaf_reg=1,
#                      scale_pos_weight=0.1,
# #                          auto_class_weights ='Balanced',
# #                       class_weights={0:0.239947,1:0.760053},
# #                       min_data_in_leaf=5,
# #                       auto_class_weights='Balanced',
#                   one_hot_max_size=10)
# m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_val, y_val)], early_stopping_rounds=50,verbose=500)
# p=m.predict_proba(X_val)

# m=OneVsRestClassifier(LGBMClassifier(n_estimators=5000,random_state=1994,learning_rate=0.05,colsample_bytree=0.1,reg_alpha=5,reg_lambda =10))
# # m=RidgeCV(cv=4)
# m.fit(X_train,y_train)
# p=m.predict_proba(X_val)





# m=RandomForestClassifier(random_state=1994,n_estimators=200)

# m.fit(X_train, y_train)
# p = m.predict_proba(X_val)
# print(f"log_loss is: {log_loss(y_val,p)}")



m=OneVsRestClassifier(LGBMClassifier())

m.fit(X_train, y_train)
p = m.predict_proba(X_val)
# print(f"log_loss is: {log_loss(y_val,p)}")

In [35]:
p

array([[7.03222400e-05, 4.60475310e-05, 7.79301525e-03, ...,
        3.06800719e-01, 1.90120409e-03, 9.99325572e-01],
       [6.64520514e-03, 7.15984972e-05, 9.28188135e-03, ...,
        2.08120944e-01, 4.82346507e-02, 1.98354881e-01],
       [5.08841280e-03, 3.96831442e-04, 1.75617190e-02, ...,
        8.15216012e-01, 5.53687360e-01, 9.93116760e-01],
       ...,
       [4.28865029e-05, 8.55269478e-05, 1.80036765e-02, ...,
        6.56761966e-02, 9.91998639e-01, 9.98630440e-01],
       [3.46010536e-04, 1.17699119e-05, 1.03099933e-03, ...,
        1.92117750e-02, 3.64947144e-02, 9.97675104e-01],
       [1.48734357e-02, 1.39254878e-05, 2.97506379e-02, ...,
        9.36761046e-01, 9.58291708e-01, 9.92350554e-01]])

In [36]:
pred = pd.DataFrame(p,index=y_val.index)
pred.describe()

,0,1,2,3,4,5,6,7,8,9,10,11
count,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000
mean,0.014958,0.011456,0.083710,0.087962,0.028522,0.431230,0.117099,0.008040,0.129905,0.339912,0.180332,0.794864
std,0.062928,0.084135,0.214821,0.216765,0.104716,0.379659,0.297579,0.070490,0.310024,0.366407,0.325609,0.326696
min,0.000043,0.000003,0.000398,0.000415,0.000058,0.003027,0.000013,0.000002,0.000081,0.004600,0.000674,0.001842
25%,0.000393,0.000034,0.003477,0.004886,0.000616,0.061396,0.000337,0.000014,0.001311,0.050558,0.005927,0.760233
50%,0.001282,0.000103,0.008781,0.011330,0.001938,0.291229,0.000985,0.000036,0.002908,0.138314,0.016805,0.975174
75%,0.005316,0.000401,0.033937,0.036044,0.007241,0.862420,0.006310,0.000117,0.010520,0.722764,0.123364,0.995974
max,0.888135,0.993057,0.998848,0.998384,0.944914,0.996869,0.999610,0.990015,0.999692,0.997466,0.999590,0.999792


In [37]:
# pred = pd.DataFrame(index=y_val.index)
# for i in range(len(p)):
    
#     pred[i] = p[i][:,-1]
# pred

In [38]:
print(f"log_loss is: {log_loss(y_val,pred)}")

log_loss is: 3.474758978507328


In [39]:
# test

In [40]:
!pip install iterative-stratification

In [41]:
from sklearn.model_selection import KFold,StratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
fold=MultilabelStratifiedKFold(n_splits=10,shuffle=True,random_state=1994)
i=1
y_pred_tot=[]
err=[]
for train_index, test_index in fold.split(X,y):
    #     X_train, X_test = X[feat_cols].iloc[train_index], X[feat_cols].iloc[test_index]
#     y_train, y_test = y[train_index], y[test_index]
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    print(X_train.shape,X_test.shape)
    model=OneVsRestClassifier(LGBMClassifier(random_state=1994))
    model.fit(X_train, y_train)
    preds_val = model.predict_proba(X_test)
    
    pred = pd.DataFrame(preds_val,index=y_test.index)
#     for i in range(len(preds_val)):

#         pred[i] = preds_val[i][:,-1]
    
    print("Log loss",log_loss(y_test,pred))
    err.append(log_loss(y_test,pred))
    test_preds = model.predict_proba(test_X)
    pred_main = pd.DataFrame(test_preds,index=test.index)
#     for i in range(len(test_preds)):
#         pred_main[i] = test_preds[i][:,-1]
    y_pred_tot.append(pred_main)

(5530, 2528) (606, 2528)
Log loss 3.598891612598026
(5512, 2528) (624, 2528)
Log loss 3.4796155890103773
(5511, 2528) (625, 2528)
Log loss 3.3702624160049615
(5524, 2528) (612, 2528)
Log loss 3.479072317575846
(5531, 2528) (605, 2528)
Log loss 3.5274687313947015
(5515, 2528) (621, 2528)
Log loss 3.5144646573776455
(5534, 2528) (602, 2528)
Log loss 3.6401109712973443
(5512, 2528) (624, 2528)
Log loss 3.325044307305318
(5517, 2528) (619, 2528)
Log loss 3.317394378778902
(5538, 2528) (598, 2528)
Log loss 3.6413272075530942


In [42]:
np.mean(err)

3.489365218889622

In [43]:
np.mean(y_pred_tot,0)

array([[4.00390162e-01, 8.62645961e-05, 1.50550397e-02, ...,
        5.93692837e-01, 6.95600953e-03, 2.01640534e-02],
       [1.42470376e-02, 3.01155615e-04, 6.02134128e-03, ...,
        1.88408995e-01, 5.49803176e-03, 8.83652267e-01],
       [1.44565862e-03, 8.23489946e-05, 1.16836976e-02, ...,
        5.82242444e-02, 9.98758794e-01, 9.94104172e-01],
       ...,
       [3.30624556e-03, 8.78663520e-03, 7.74991364e-04, ...,
        9.81009195e-01, 1.97893215e-03, 9.50886544e-01],
       [4.12260841e-04, 4.13048179e-04, 7.54973462e-01, ...,
        9.96668504e-01, 3.71093949e-02, 9.20191557e-01],
       [2.68003749e-03, 1.81167749e-05, 1.64226376e-02, ...,
        5.54972173e-02, 6.06049483e-01, 9.78632805e-01]])

In [44]:
pred_main = pd.DataFrame(data=np.mean(y_pred_tot,0),index=test.index)
# for i in range(len(y_main_pred)):
    
#     pred_main[i] = y_main_pred[i][:,-1]

In [45]:
pred_main.columns = target
pred_main

,Components,Delivery and Customer Support,Design and Aesthetics,Dimensions,Features,Functionality,Installation,Material,Price,Quality,Usability,Polarity
0,0.400390,0.000086,0.015055,0.009172,0.001165,0.120714,0.000306,0.005439,0.004084,0.593693,0.006956,0.020164
1,0.014247,0.000301,0.006021,0.012896,0.003832,0.878637,0.000488,0.000012,0.006083,0.188409,0.005498,0.883652
2,0.001446,0.000082,0.011684,0.016874,0.010572,0.078672,0.003667,0.000006,0.081258,0.058224,0.998759,0.994104
3,0.001768,0.001695,0.003605,0.019165,0.009052,0.942489,0.001189,0.000021,0.006986,0.104578,0.003167,0.962892
4,0.002126,0.000663,0.002882,0.015066,0.000655,0.486193,0.000559,0.000038,0.001314,0.232427,0.005725,0.956074
...,...,...,...,...,...,...,...,...,...,...,...,...
2626,0.000910,0.000045,0.005803,0.008409,0.118331,0.076657,0.850911,0.000068,0.002287,0.798589,0.295524,0.992398
2627,0.077054,0.000208,0.023502,0.022769,0.002301,0.123646,0.000249,0.000020,0.003821,0.066689,0.018284,0.839603
2628,0.003306,0.008787,0.000775,0.002477,0.000295,0.469640,0.000702,0.000070,0.001037,0.981009,0.001979,0.950887
2629,0.000412,0.000413,0.754973,0.035694,0.007843,0.015745,0.000228,0.000428,0.008134,0.996669,0.037109,0.920192


In [46]:
test['Review']

0       Made of very thin cheap metal broke on very fi...
1       As good as the brand names, no jams or misfire...
2       unit was easy to use, with understandable in s...
3       I am the new family plumber. Works well. No pr...
4                            Seems to be holding up well.
                              ...                        
2626    Very strong piece of hardware. Easy to adjust ...
2627    Great supot.  Square Wich is better to line up...
2628             No jams, no problems. Good quality nail!
2629    Chair cushion is and very firm, however nice r...
2630    Easy to work with and install. We use this for...
Name: Review, Length: 2631, dtype: object

In [47]:
pred_main.to_csv('MH_uhack_s5.csv',index=False)